In [ ]:
#@title Запуск TerraAI

import os
from time import sleep
from enum import Enum
from pathlib import Path
from typing import Tuple
from ipywidgets import widget, Text, Password, Button, VBox, HBox, Layout


TERRA_REPOSITORY = "https://github.com/aiuniver/terra_gui.git"
TERRA_SERVER_API = "http://%sterra.neural-university.ru/api/v1"
TERRA_DIRECTORY = "terra"


class WebServerException(Exception):
    pass


class State(str, Enum):
    idle = "idle"
    launch = "launch"
    restore_token = "restore_token"


def show_exception(message):
    print(f"\033[1;31m{str(message)}\033[0m")


def handler_change_field(field):
    ws.gui.launch_button.disabled = not (ws.gui.email_field.value and ws.gui.token_field.value)
    ws.gui.restore_token_button.disabled = not ws.gui.email_field.value


def handler_start_launch(button):
    if not (ws.gui.email_field.value and ws.gui.token_field.value):
        return
    ws.gui.state = State.launch
    try:
        ws.start()
    except WebServerException as error:
        show_exception(error)
        ws.gui.state = State.idle
        ws.gui.launch_button.disabled = not (ws.gui.email_field.value and ws.gui.token_field.value)


def handler_stop_launch(button):
    ws.gui.state = State.idle
    try:
        ws.stop()
    except WebServerException as error:
        show_exception(error)
        ws.gui.state = State.idle
        ws.gui.launch_button.disabled = not (ws.gui.email_field.value and ws.gui.token_field.value)


def handler_restore_token(button):
    ws.gui.state = State.restore_token
    try:
        ws.restore_token()
        print(f"\033[1;32mНа E-mail {ws.gui.email_field.value} отправлено письмо с новым токеном\033[0m")
    except WebServerException as error:
        show_exception(error)
    ws.gui.state = State.idle


class WebServerGUI:
    email_field: Text
    token_field: Password
    launch_button: Button
    stop_button: Button
    restore_token_button: Button

    def __init__(self):
        self.email_field = Text(placeholder="Введите E-mail", disabled=True, layout=Layout(width="300px"))
        self.token_field = Password(placeholder="Введите Token", disabled=True, layout=Layout(width="300px"))
        self.launch_button = Button(description="Запустить", disabled=True, layout=Layout(width="148px"))
        self.stop_button = Button(description="Остановить", disabled=True, layout=Layout(width="148px"))
        self.restore_token_button = Button(description="Восстановить токен", disabled=True, layout=Layout(width="300px"))

        self.email_field.on_submit(handler_start_launch)
        self.email_field.observe(handler_change_field, names=["value"])

        self.token_field.on_submit(handler_start_launch)
        self.token_field.observe(handler_change_field, names=["value"])

        self.launch_button.on_click(handler_start_launch)
        self.stop_button.on_click(handler_stop_launch)
        self.restore_token_button.on_click(handler_restore_token)

    @property
    def state(self) -> Tuple[bool, bool, bool, bool]:
        return (
            not self.email_field.disabled,
            not self.token_field.disabled,
            not self.launch_button.disabled,
            not self.stop_button.disabled,
            not self.restore_token_button.disabled,
        )

    @state.setter
    def state(self, value:State):
        if value == State.idle:
            self.email_field.disabled = False
            self.token_field.disabled = False
            self.launch_button.disabled = not (self.email_field.value and self.token_field.value)
            self.stop_button.disabled = True
            self.restore_token_button.disabled = not self.email_field.value
        elif value == State.restore_token:
            self.email_field.disabled = True
            self.token_field.disabled = True
            self.launch_button.disabled = True
            self.stop_button.disabled = True
            self.restore_token_button.disabled = True
        elif value == State.launch:
            self.email_field.disabled = True
            self.token_field.disabled = True
            self.launch_button.disabled = True
            self.stop_button.disabled = False
            self.restore_token_button.disabled = True

    def display(self):
        self.state = State.idle
        widget.display(
            VBox(
                [
                    self.email_field,
                    self.token_field,
                    HBox([self.launch_button, self.stop_button]),
                    self.restore_token_button,
                ]
            )
        )


class WebServer:
    _branch:str = "dev"
    _domain_prefix:str = "dev."
    _path:Path = Path(os.path.abspath(os.getcwd()))
    _url:str = ""
    _data:dict = {}
    _proc:str

    gui:WebServerGUI = WebServerGUI()

    def _auth(self):
        try:
            response = requests.post(
                f"{TERRA_SERVER_API % self._domain_prefix}/login/",
                json={"email": self.gui.email_field.value, "user_token": self.gui.token_field.value},
            )
        except Exception as error:
            raise WebServerException(error)

        if not response.ok:
            try:
                response.raise_for_status()
            except requests.exceptions.HTTPError as error:
                raise WebServerException(error)

        data = response.json()
        if not data.get("success"):
            raise WebServerException(data.get("error"))

        self._url = data.get("data", {}).get("url", "")
        if not self._url:
            raise WebServerException("Не определен URL для пользовательского интерфейса")

        self._data = data.get("data", {})

    def _download(self):
        _terra_path = Path(self._path, TERRA_DIRECTORY)
        _repo_kwargs = {}
        if self._branch:
            _repo_kwargs.update({"branch": self._branch})
        shutil.rmtree(_terra_path, ignore_errors=True)
        try:
            Repo.clone_from(TERRA_REPOSITORY, _terra_path, **_repo_kwargs)
        except Exception as error:
            raise WebServerException(error)

    def _prepare(self):
        _terra_path = Path(self._path, TERRA_DIRECTORY)
        for name, info in self._data.get("create", {}).items():
            with open(Path(_terra_path, info.get("name")), "w") as _file_ref:
                _file_ref.write(info.get("data"))

        with open(Path(self._path, TERRA_DIRECTORY, ".env"), "a") as _env_file_ref:
            _env_file_ref.write(f"USER_URL={self._url}\n")
            _env_file_ref.write(f"USER_EMAIL={self.gui.email_field.value}\n")
            _env_file_ref.write(f"USER_TOKEN={self.gui.token_field.value}\n")

    def init(self):
        self.gui.display()

    def start(self):
        self._auth()
        self._download()
        self._prepare()
        print(f"\033[1;32mДля начала работы перейдите по следующей ссылке {self._url}\033[0m")
        with open(Path(self._path, TERRA_DIRECTORY, "logs.txt"), "w") as logs_file_ref:
            self._proc = Popen(["make", "-C", Path(self._path, TERRA_DIRECTORY)], stdin=PIPE, stdout=logs_file_ref, stderr=STDOUT, preexec_fn=os.setsid)

    def stop(self):
        if not self._proc:
            return
        os.killpg(os.getpgid(self._proc.pid), signal.SIGTERM)

    def restore_token(self):
        try:
            response = requests.post(
                f"{TERRA_SERVER_API % self._domain_prefix}/restore-token/",
                json={"email": self.gui.email_field.value},
            )
        except Exception as error:
            raise WebServerException(error)

        if not response.ok:
            try:
                response.raise_for_status()
            except requests.exceptions.HTTPError as error:
                raise WebServerException(error)

        data = response.json()
        if not data.get("success"):
            raise WebServerException(data.get("error"))


from google.colab import drive, output

%cd /content
output.clear()

mounted = False
try:
    drive.mount("/content/drive")
    mounted = True
except ValueError as error:
    output.clear()
    show_exception("Не удалось подключиться к GoogleDrive")
except KeyboardInterrupt as error:
    pass

ws = WebServer()

if mounted:
    output.clear()
    print("Начало установки. Процесс может занять несколько минут, ожидайте...")
    !pip install git+https://github.com/aiuniver/terra_colab.git@dev &> /dev/null
    print("Установка завершена!")
    sleep(1)
    output.clear()

    import shutil
    import signal
    import requests
    from git import Repo
    from subprocess import Popen, PIPE, STDOUT

    try:
        ws.init()
    except WebServerException as error:
        show_exception(error)
        ws.gui.state = State.idle
